# Hello Blqs_cirq

In this notebook you will find a simple explanation for the motivation for blqs_cirq.  First make sure blqs_cirq is installed (it will install blqs and cirq as dependencies).

In [6]:
try:
    import blqs_cirq
except ImportError:
    print("installing blqs_cirq...")
    !pip install --quiet blqs
    print("installed blqs_cirq.")

import blqs_cirq as bc
import blqs
import cirq    

This is an example of how to create a circuit using Cirq.  One sees that one has a base object upon which you append the elements.

In [7]:
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit()
circuit.append(cirq.H(q0))
circuit.append([cirq.H(q1), cirq.CX(q0, q1)])
print(circuit)

0: ───H───@───
          │
1: ───H───X───


Blqs_cirq allows one to do this in a more imperative manner.

In [9]:
from blqs_cirq import H, CX

@bc.build
def my_circuit():
    H(0)
    H(1)
    CX(0, 1)
    
circuit = my_circuit()
print(circuit)

0: ───H───@───
          │
1: ───H───X───


The object we produce is a `cirq.Circuit`

In [10]:
type(circuit)

cirq.circuits.circuit.Circuit

Notice that we have used the decorator `bc.build` on the method. This turns the method into a builder of `cirq.Circuit`s. 

Second notice that we have used the `blqs_cirq` version of the cirq gates (every gate in cirq exists in blqs_cirq) as the statements in our method.

Finally notice that we have used integers instead of `cirq.LineQubit`s. Blqs_cirq automatically converts `int`s to `cirq.LineQubit`s with the int as the coordinate.

Blqs_cirq makes it easier to write imperative cirq circuits. As cirq adds control statement, it will also make it easier to write these in native python.  An example of that which exists today is for Cirq's `CircuitOperation`.  A `CircuitOperation` in cirq is an operation on qubits that represents a subcircuit. In particular it is a subcircuit that can be repeated.

In cirq you might do something like

In [11]:
sub_circuit = cirq.Circuit()
sub_circuit.append(cirq.H(q0))
sub_circuit.append(cirq.CX(q0, q1))

circuit = cirq.Circuit()
circuit.append(cirq.H(q0))
circuit.append(cirq.CircuitOperation(sub_circuit.freeze(), repetitions=10))

print(circuit)

          Circuit_0xd37291915e463e42:
0: ───H───[ 0: ───H───@───          ]─────────────
          [           │             ]
          [ 1: ───────X───          ](loops=10)
          │
1: ───────#2──────────────────────────────────────


In blqs_cirq we can do the same thing using the `Repeat` context manager.

In [14]:
@bc.build
def my_circuit():
    H(0)
    with bc.Repeat(repetitions=10):
        H(0)
        CX(0, 1)

circuit = my_circuit()
print(circuit)

          Circuit_0xd37291915e463e42:
0: ───H───[ 0: ───H───@───          ]─────────────
          [           │             ]
          [ 1: ───────X───          ](loops=10)
          │
1: ───────#2──────────────────────────────────────


As more control flow is added to Cirq, eventually these will end up in blqs_cirq as python native constructs.

To learn more, see the [introduction to blqs_cirq](docs/intro.md)